# Visualize and Analyze Data

This notebook will explore the data we downloaded in the `Customize and Access Data` notebook to fulfill the final objective of this tutorial: 

* Interact with ICESat-2 and MODIS data using basic Python science libraries to visualize, filter, and plot concurrent data.

We will perform the following steps:

1. Read in ATL07 data
2. Convert ATL07 delta_time to UTC time
3. Compute ATL07 along track distance per beam
4. Read in MOD29 data and apply scale factor 
5. Overlay ATL07 data over MODIS IST
3. Extract MODIS IST values along ICESat-2 heights
4. Extract ICESat-2 heights along MODIS IST values.
5. Plot MODIS IST and ICESat-2 heights
   

### Import packages

In [ ]:
from collections import defaultdict
from osgeo import gdal, osr
import math
import h5py
import xarray as xr
import pyproj
import pandas as pd
from shapely.geometry import Point,Polygon
import pyresample as prs
import cartopy.crs as ccrs
from matplotlib import pyplot as plt
import hvplot.pandas  # Adds the 'hvplot' method to pandas objects
import hvplot.xarray  # Adds the 'hvplot' method to xarray objects
import rasterio
import numpy as np
from holoviews.operation.datashader import datashade
import datashader.geo

# Helper functions
import tutorial_helper_functions as fn

%matplotlib inline

In [ ]:
!aws --no-sign-request --region=us-west-2 s3 sync s3://nsidc-agu-tutorial-data/ Outputs

## Read in ATL07 and perform calculations

We will be using a few different packages to read in our data:

* [Pandas DataFrame](https://pandas.pydata.org/pandas-docs/stable/getting_started/overview.html)
    * Pandas is built on top of NumPy and provides easy to work with data structures. The pandas dataframe is a 2-D tabular data structure with labeled axes. 
* [Gdal](https://pypi.org/project/GDAL/)
    * Tools for programming and manipulating the Geospatial Data Abstraction Library (GDAL).
* [Rasterio](https://rasterio.readthedocs.io/en/stable/index.html)
    * Rasterio reads and writes raster formats including GeoTIFF.
* [Xarray](http://xarray.pydata.org/en/stable/index.html)
    * Designed for multidimensional data. Xarray introduces labels in the form of dimensions, coordinates and attributes on top of raw NumPy-like arrays. 

### Load ATL07 granules into dataframes

We will start by defining variables to extract and combine as a single dataframe:

In [ ]:
VARIABLES = {
    'ATL07': [
        '/gt1l/sea_ice_segments/delta_time',
        '/gt1l/sea_ice_segments/latitude',
        '/gt1l/sea_ice_segments/longitude',
        '/gt1l/sea_ice_segments/heights/height_segment_confidence',
        '/gt1l/sea_ice_segments/heights/height_segment_height',
        '/gt1l/sea_ice_segments/heights/height_segment_quality',
        '/gt1l/sea_ice_segments/heights/height_segment_surface_error_est',
        '/gt1l/sea_ice_segments/heights/height_segment_length_seg',
        '/gt2l/sea_ice_segments/delta_time',
        '/gt2l/sea_ice_segments/latitude',
        '/gt2l/sea_ice_segments/longitude',
        '/gt2l/sea_ice_segments/heights/height_segment_confidence',
        '/gt2l/sea_ice_segments/heights/height_segment_height',
        '/gt2l/sea_ice_segments/heights/height_segment_quality',
        '/gt2l/sea_ice_segments/heights/height_segment_surface_error_est',
        '/gt2l/sea_ice_segments/heights/height_segment_length_seg',
        '/gt3l/sea_ice_segments/delta_time',
        '/gt3l/sea_ice_segments/latitude',
        '/gt3l/sea_ice_segments/longitude',
        '/gt3l/sea_ice_segments/heights/height_segment_confidence',
        '/gt3l/sea_ice_segments/heights/height_segment_height',
        '/gt3l/sea_ice_segments/heights/height_segment_quality',
        '/gt3l/sea_ice_segments/heights/height_segment_surface_error_est',
        '/gt3l/sea_ice_segments/heights/height_segment_length_seg'
    ]
}

#### We will use the `load_icesat2_as_dataframe` function from our functions module to load points from our ATL07 files: 

In [ ]:
# Load ATL07 granules and combine into single ATL07 dataframe

atl07_1298 = fn.load_icesat2_as_dataframe('./Outputs/processed_ATL07-01_20190323101341_12980201_002_01.h5', VARIABLES)
atl07_1305 = fn.load_icesat2_as_dataframe('./Outputs/processed_ATL07-01_20190323211343_13050201_002_01.h5', VARIABLES)

atl07 = pd.concat([atl07_1298,atl07_1305]) #concatenate dataframes to use in plot below

atl07_1305.head()

### Convert ATL07 'delta_time' to utc datetime values

In [ ]:
# Add datetime values to each dataframe

data_granules = [atl07_1298, atl07_1305]

for df in data_granules:
    df['utc_datetime'] = df['delta_time'].apply(fn.convert_delta_time)
    df = df.sort_values(by=['beam', 'utc_datetime'])

# atl07_1298
data_granules[0].head()

### Compute along track distance

The `height_segment_length_seg` variable provides the length of each segment. We need these values to be additive so that we can plot against distance.  


In [ ]:
from IPython.display import display, HTML

for df in data_granules:
    df['add_dist'] = df.groupby(['beam'])['height_segment_length_seg'].cumsum()
    

# same as data_granules[0]
display(HTML(atl07_1298.loc[atl07_1298['beam'] == 'gt1l'].head().to_html()))


## Read in MODIS Ice Surface Temperature file and perform calculations

There were 13 granules returned over our time and area of interest. We can use Earthdata Search to easily visualize the coverage of each granule:
[MODIS Earthdata Search link](https://search.earthdata.nasa.gov/search/granules?p=C1000001160-NSIDC_ECS&q=mod29&sb=140%2C72%2C153%2C80&m=80.14349326047419!-42.27686414753069!0!0!0!0%2C2&qt=2019-03-23T00%3A00%3A00.000Z%2C2019-03-23T23%3A59%3A59.999Z&tl=1557787959!4!)

Let's choose a single MODIS granule to be used in our analysis below. 

Out of the 13 returned, these all have full coverage over our study area:

* MOD29.A2019082.0810.006.2019082191746.hdf
* MOD29.A2019082.0630.006.2019082191633.hdf
* MOD29.A2019082.0455.006.2019082132032.hdf
* MOD29.A2019082.0945.006.2019082191513.hdf

And after viewing the browse images, this looks to have the best data coverage:

* MOD29.A2019082.0140.006.2019082132036.hdf


### Load the MOD29 file into xarray

In [ ]:
# Define local filepath and open as xarray

filepath = './Outputs/MOD29_A2019082_0140_006_2019082132036_HEGOUT.hdf'

mod29 = xr.open_dataset(filepath)
mod29

### Convert calibrated data to Kelvin

Before we plot the ATL07 heights over MODIS IST, we need to convert the Ice_Surface_Temperature values to kelvin. According to the [MOD29 User Guide](https://nsidc.org/data/mod29#title11) the Ice_Surface_Temperature (IST) values are stored as calibrated data. To convert to kelvin, use scale_factor = 0.01 and add_offset = 0.0 in the following equation:

IST = scale_factor × (calibrated data - add_offset)

The valid range for ISTs is 210 K to 313.20 K.

Let's apply this conversion to our xarray:

In [ ]:
mod29['Ice_Surface_Temperature'] = mod29['Ice_Surface_Temperature']*(0.01) # Apply calibrated data to kelvin conversion

### Load MODIS IST image

A GeoTIFF was created using the NASA Worldview snapshot feature from MO29 high-resolution browse imagery: </n> 

MODIS IST: https://go.nasa.gov/34fBnp8

We'll read in the projection information below to help inform our plotting projection parameters in the next step:

In [ ]:
# Read geotiff images from gdal 
gdal.UseExceptions()

fname_ist = './Outputs/snapshot-2019-03-23T00_00_00Z-IST.tiff'

ds_ist = gdal.Open(fname_ist)
proj_ist = ds_ist.GetProjection()
inproj = osr.SpatialReference()
inproj.ImportFromWkt(proj_ist)

#Print proj4 string
proj4_ist = inproj.ExportToProj4()
print('MODIS IST proj4:',proj4_ist)

## Visualize ICESat-2 height values over MODIS Ice Surface Temperature

We can visualize the ICESat-2 track locations and height values overlayed with MODIS Ice Surface Temperature. We should see higher ATL07 heights along fast ice, with lower heights over thin ice and then increasing height over drift ice.  [YOU WILL GET A MESSAGE IF YOU FIRST DOWNLOADED THE COASTLINE] [add ist values and crop image] [CHANGE WIDTH / HEIGHTS]

In [ ]:
# Read in data using rasterio
#source_ist = xr.open_rasterio(fname_ist)
source_ist = rasterio.open(fname_ist, 'r')
bounds_ist = (source_ist.bounds.left, source_ist.bounds.right, \
          source_ist.bounds.bottom, source_ist.bounds.top)


# Luis: Can you add the color scale for the MODIS image and make the color scale look nicer?

In [ ]:
# Plot ICESat-2 over MODIS IST

# TODO
# add geotiff scale values and if possible add IS tracks to hvplots

fig = plt.figure(figsize=(8, 10))
img = plt.imread(fname_ist)
plot_proj = ccrs.Stereographic(central_longitude=-45, central_latitude=90, 
                               false_easting=0, false_northing=0,
                               true_scale_latitude=70)

ax = plt.axes(projection=plot_proj)
plt.title('ATL07 height values\n'
          '03/23/2019')

# set a margin around the data
ax.set_xmargin(0.05)
ax.set_ymargin(0.10)

# add the image. Because this image was a tiff, the "origin" of the image is in the upper left corner
base_img = ax.imshow(img, origin='upper', extent=bounds_ist, transform=plot_proj)
cbar2 = fig.colorbar(base_img, ax=ax)

cbar2.set_label('Ice Surface Temperature (K)')
ax.coastlines(resolution='50m', color='black', linewidth=1)

heights = ax.scatter(atl07.longitude, atl07.latitude, transform=ccrs.Geodetic(),
          c=atl07.height_segment_height,
          cmap='Reds', alpha=0.6)              

cbar = fig.colorbar(heights, ax=ax)

cbar.set_label('Height (m)')

plt.show()

# Luis: Another plotting option is to use hvplot to plot the MODIS HDF file itself from xarray and plot the ATL07 points over it. I had trouble doing this, however.

In [ ]:
# title = 'MOD29 Ice Surface Temperature (K)'
rasterized_ist = mod29.Ice_Surface_Temperature.hvplot.image(rasterize=True, 
                                         cmap='viridis', height=400, width=800).redim.range(Ice_Surface_Temperature=(240,270))
rasterized_ist.options(cmap='viridis')


#Incorporate ICESat-2 values
# datashade(df_atl07.hvplot.points(x='longitude', y='latitude'))
# x, y = datashader.geo.lnglat_to_meters(df_atl07.longitude, df_atl07.latitude)

# mod29.Ice_Surface_Temperature.hvplot.quadmesh('Along_swath_lines_1km:MOD_Swath_Sea_Ice', 'Cross_swath_pixels_1km:MOD_Swath_Sea_Ice', projection=test, project=True, 
#                                               cmap='viridis', rasterize=True, dynamic=False,).redim.range(Ice_Surface_Temperature=(240,700))

## Extract MODIS IST values along the ICESat-2 tracks

Now that we've visually inspected height and IST values, let's do some analysis. We will now extract MODIS IST values at each ATL07 point.


### Interpolating MOD29 onto ICESat-2 points

To easily interpolate MOD29 to ICESat-2 points, we can use the `pyresample` library. The `radius_of_influence` parameter determines maximum radius to look for nearest neighbor interpolation.

In [ ]:
icesat2_geometry_1298 = prs.geometry.SwathDefinition(lons=atl07_1298['longitude'], lats=atl07_1298['latitude'])
icesat2_geometry_1305 = prs.geometry.SwathDefinition(lons=atl07_1305['longitude'], lats=atl07_1305['latitude'])

mod29_geometry = prs.geometry.SwathDefinition(lons=mod29['Longitude'], lats=mod29['Latitude'])
atl07_1298['mod29_ice_surface_temperature'] = prs.kd_tree.resample_nearest(mod29_geometry, mod29['Ice_Surface_Temperature'].values, icesat2_geometry_1298, radius_of_influence=1000, fill_value=np.nan)
atl07_1305['mod29_ice_surface_temperature'] = prs.kd_tree.resample_nearest(mod29_geometry, mod29['Ice_Surface_Temperature'].values, icesat2_geometry_1305, radius_of_influence=1000, fill_value=np.nan)

# Luis: Having trouble with loc key

In [ ]:
# apply nan's to MOD29 land mask

# need to use .loc method

mask_1298 = atl07_1298['mod29_ice_surface_temperature'] == 50
atl07_1298['mod29_ice_surface_temperature'][mask_1298] = np.nan

mask_1305 = atl07_1305['mod29_ice_surface_temperature'] == 50
atl07_1305['mod29_ice_surface_temperature'][mask_1305] = np.nan

## Plot MODIS IST and ICESat-2 height along tracks

* This should show cold IST over the fast ice with fairly flat and higher ICESat-2 heights
* Warm IST will be seen in open water, thin ice just off the fast ice boundary
    * IST will decrease along transect
    * IS-2 should show low heights, but probably low quality (at least for ATL10, if any are available)
    
 * Over fast ice: IST should be close to constant and height will also be fairly constant
* Over drift ice, where it goes from thin ice toward thicker ice, we should see height increasing as IST decreases
* If we use IST = 271.2 K for open water and then find the IST at the first valid ATL10 value, we have the IST value at that freeboard. Then can use that relationship to get thin ice freeboard where there is ATL07 data, but not ATL10
* Comparing data with and without quality flag, the high temps are getting filtered out - more open water.
    
 [JUST PLOT ONE BEAM AT A TIME WITH DISTANCE]
 [ATL07 VS ATL10]
 [PLOT THOSE QUALITY VALUES - INTERACTIVE]
 [PLOT ATL10 QUALITY FLAGS LOWER ON ATL07]
 [TRY DIFFERENT BEAM]
 [interactive: like plotting each of the quality flags and comparing to the height values]
 
 
 [dont need GEOPANDAS - JUST USE PANDAS]
 [SAME PLOT MAP TOOLS]
 1. SHOW MAP, 2. SHOW LAYERS, 3. XY PLOT, 
 

### First, we'll plot against time.

ATL07-01_20190323211343_13050201_002_01 is moving from low to high latitude: The start time begins at the fast ice, skips over an island, and moves towards warmer thin ice onto thicker drift ice.

ATL07-01_20190323101341_12980201_002_01 is moving from high to low latitude: The start time begins at the thicker drift ice, moving towards thinner ice and open water, ending at colder fast ice. 

In [ ]:
# Using pandas plotting (matplotlib backend)
fig, ax = plt.subplots(figsize=(10,3))
atl07_1305.plot(x='utc_datetime', y='height_segment_height', ax=ax)
atl07_1305.plot(x='utc_datetime', y='mod29_ice_surface_temperature', ax=ax, secondary_y=True)
plt.title('ATL07-01_20190323211343_13050201_002_01');

fig, ax = plt.subplots(figsize=(10,3))
atl07_1298.plot(x='utc_datetime', y='height_segment_height', ax=ax)
atl07_1298.plot(x='utc_datetime', y='mod29_ice_surface_temperature', ax=ax, secondary_y=True)
plt.title('ATL07-01_20190323101341_12980201_002_01');

### Next, let's plot against distance. 

Because each height segment represents a variable distance based on 150 photon returns, this is a more accurate representation of height and IST values across the transect. We'll focus on ICESat-2 track 1298 since this track has fewer data gaps over the islands.

In [ ]:
fig, ax = plt.subplots(figsize=(10,3))
fig.suptitle('ATL07-01_20190323101341_12980201_002_01 gt1l')
atl07_1298.loc[atl07_1298['beam'] == 'gt1l'].plot(x='add_dist', y='height_segment_height', ax=ax)
atl07_1298.loc[atl07_1298['beam'] == 'gt1l'].plot(x='add_dist', y='mod29_ice_surface_temperature', ax=ax, secondary_y=True)

fig, ax = plt.subplots(figsize=(10,3))
fig.suptitle('ATL07-01_20190323101341_12980201_002_01 gt2l')
atl07_1298.loc[atl07_1298['beam'] == 'gt2l'].plot(x='add_dist', y='height_segment_height', ax=ax)
atl07_1298.loc[atl07_1298['beam'] == 'gt2l'].plot(x='add_dist', y='mod29_ice_surface_temperature', ax=ax, secondary_y=True)

fig, ax = plt.subplots(figsize=(10,3))
fig.suptitle('ATL07-01_20190323101341_12980201_002_01 gt3l')
atl07_1298.loc[atl07_1298['beam'] == 'gt3l'].plot(x='add_dist', y='height_segment_height', ax=ax)
atl07_1298.loc[atl07_1298['beam'] == 'gt3l'].plot(x='add_dist', y='mod29_ice_surface_temperature', ax=ax, secondary_y=True);


### Data filtering by quality flag

We'll compare data before and after applying the quality flags we subsetted. Let's start by filtering `/sea_ice_segments/heights/height_segment_height` based on `/sea_ice_segments/heights/height_segment_quality` for each beam group. According to the [User Guide](https://nsidc.org/data/ATL07#title38):

height_segment_quality: a binary indicator (1 = good, 0 = bad) of segment quality

In [ ]:
cond = {'beam': 'gt1l', 'height_segment_quality': 1}
fig, ax = plt.subplots(figsize=(10,3))
atl07_1298.loc[(atl07_1298.beam == cond['beam']) & (atl07_1298.height_segment_quality == cond['height_segment_quality'])].plot(x='add_dist', y='height_segment_height', ax=ax)
atl07_1298.loc[(atl07_1298.beam == cond['beam']) & (atl07_1298.height_segment_quality == cond['height_segment_quality'])].plot(x='add_dist', y='mod29_ice_surface_temperature', ax=ax, secondary_y=True)
plt.title('ATL07-01_20190323211343_13050201_002_01 gt1l filtered');

In [ ]:
atl07_1298[qual_filter_1298].plot(x='utc_datetime', y='height_segment_height', ax=ax)
atl07_1298[qual_filter_1298].plot(x='utc_datetime', y='mod29_ice_surface_temperature', ax=ax, secondary_y=True)
plt.title('ATL07-01_20190323211343_13050201_002_01 filtered');



In [ ]:
datashade(df_07_101341[qual_filter_101341].hvplot.points(x='mod29_ice_surface_temperature', y='height_segment_height', c='latitude', height=800, width=2000))

In [ ]:
atl07_frames = [df_07_101341, df_07_211343]
df_atl07 = pd.concat(atl07_frames,ignore_index=True)
datashade(df_atl07.hvplot.points(x='mod29_ice_surface_temperature', y='height_segment_height', c='latitude', height=800, width=2000))

In [ ]:
datashade(df_07_211343[qual_filter_211343].hvplot.points(x='mod29_ice_surface_temperature', y='height_segment_height', c='latitude', size=400))

Add ATL10 to show coverage and freeboard estimate.

## Extract ICESat-2 heights along MODIS IST values

* Use MODIS footprint as the baseline and match/bin IS-2 to each MODIS observation

Extra code...

In [ ]:
# # separate each beam

# gt1l = df_single['beam']=='gt1l'
# gt2l = df_single['beam']=='gt2l'
# gt3l = df_single['beam']=='gt3l'

In [ ]:
# fig, ax = plt.subplots(figsize=(15, 5))
# df_single[gt1l].plot(x='distance', y='height_segment_height', ax=ax)
# df_single[gt1l].plot(x='distance', y='mod29_ice_surface_temperature', ax=ax, secondary_y=True)

# fig, ax = plt.subplots(figsize=(15, 5))
# df_single[gt2l].plot(x='distance', y='height_segment_height', ax=ax)
# df_single[gt2l].plot(x='distance', y='mod29_ice_surface_temperature', ax=ax, secondary_y=True)

# fig, ax = plt.subplots(figsize=(15, 5))
# df_single[gt3l].plot(x='distance', y='height_segment_height', ax=ax)
# df_single[gt3l].plot(x='distance', y='mod29_ice_surface_temperature', ax=ax, secondary_y=True)

In [ ]:
df_single[qual_filter].plot.scatter(x='mod29_ice_surface_temperature', y='height_segment_height')



In [ ]:
## add ATL10 now? This could show coverage and show the freeboard estimate. 

### A couple other possibilities:
* Compare cross-over points between tracks 1298 and 1305. 
    * This is over drift ice, but might be able to correlate between them
    * Using all strong beams, there are 9 cross-over points, so there is 
* Compare 1298 repeat track from 12/22/2018 and 3/23/2019
    * For fast ice, should see an increase in height from Dec to Mar as snow depth and ice thickness increases
        * From a quick look at OA, I’m not really seeing this though
        * Not much from ATL10
        * Lots of data in ATL07, but don’t really see a connection – in fact Dec looks lower than Mar; this could be due to corrections/quality issues

In [ ]:
# df_07_101341 = pd.DataFrame(gdf_07_101341.drop(columns='geometry'))
# df_07_211343 = pd.DataFrame(gdf_07_211343.drop(columns='geometry'))

#### Explore ATL03 (photon-level) heights

Load ATL03 Variables:

In [ ]:
# VARIABLES = {
#     'ATL03': [
#         '/gt1l/heights/delta_time',
#         '/gt1l/heights/h_ph',
#         '/gt1l/heights/lat_ph',
#         '/gt1l/heights/lon_ph',
#         '/gt1l/heights/signal_conf_ph',
#         '/gt2l/heights/delta_time',
#         '/gt2l/heights/h_ph',
#         '/gt2l/heights/lat_ph',
#         '/gt2l/heights/lon_ph',
#         '/gt2l/heights/signal_conf_ph',
#         '/gt3l/heights/delta_time',
#         '/gt3l/heights/h_ph',
#         '/gt3l/heights/lat_ph',
#         '/gt3l/heights/lon_ph',
#         '/gt3l/heights/signal_conf_ph'
#     ],

In [ ]:
# Running out of memory on these:
# #df_03_103438 = functions.load_icesat2_as_dataframe('./Outputs/processed_ATL03_20190323103438_12980204_002_01.h5', VARIABLES)
# df_03_103947 = functions.load_icesat2_as_dataframe('./Outputs/processed_ATL03_20190323103947_12980205_002_01.h5', VARIABLES)
# df_03_212915 = functions.load_icesat2_as_dataframe('./Outputs/processed_ATL03_20190323212915_13050203_002_01.h5', VARIABLES)
# #df_03_213440 = functions.load_icesat2_as_dataframe('./Outputs/processed_ATL03_20190323213440_13050204_002_01.h5', VARIABLES)


#atl03_frames = [df_03_103438, df_03_103947, df_03_212915, df_03_213440]
#atl03_df = pd.concat(atl03_frames,ignore_index=True)


In [ ]:
# min_lon, max_lon = mod29.Longitude.min(), mod29.Longitude.max()
# min_lat, max_lat = mod29.Latitude.min(), mod29.Latitude.max()
# mod29_boundary = Polygon(
#     [
#         (min_lon, min_lat), 
#         (max_lon, min_lat), 
#         (max_lon, max_lat), 
#         (min_lon, max_lat), 
#         (min_lon, min_lat)
#     ]
# )

In [ ]:
# Warning: rather slow operation here, takes a few minutes for 200000 points
#distance = gdf_all.groupby(['filename', 'beam']).apply(functions.compute_along_track_distance)

# Warning: rather slow operation here, takes a few minutes for 200000 points
#distance_single = gdf_single.groupby(['filename','beam']).apply(functions.compute_along_track_distance)

# Convert pandas series to list
#distance_list = distance_single.tolist()

# Create new gdf with distance added
# gdf_dist = gdf_single.assign(distance = distance_list)